# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
# default_exp examples

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
#export
#hide
import requests
import json
import arrow
import pandas as pd
import sys; sys.path.append('../')
from prodb.core import generate_db, insert_row

In [ ]:
#export
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]

In [ ]:
l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/12/30'

In [ ]:
#export
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast 🌤️

In [ ]:
%%time
res = get_current_weather('London')
res['consolidated_weather'][0]

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 1.48 s


{'id': 6292426705600512,
 'weather_state_name': 'Heavy Cloud',
 'weather_state_abbr': 'hc',
 'wind_direction_compass': 'SW',
 'created': '2021-12-30T12:59:01.380456Z',
 'applicable_date': '2021-12-30',
 'min_temp': 12.665,
 'max_temp': 14.465,
 'the_temp': 14.274999999999999,
 'wind_speed': 9.857931369964739,
 'wind_direction': 225.0,
 'air_pressure': 1013.0,
 'humidity': 89,
 'visibility': 7.299868766404199,
 'predictability': 71}

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,6292426705600512,Heavy Cloud,hc,SW,2021-12-30T12:59:01.380456Z,2021-12-30,12.665,14.465,14.275,9.857931,225.0,1013.0,89,7.299869,71,2 hours ago


In [ ]:
#export
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = pd.to_datetime(res['time']).strftime('%H:%M')
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]

In [ ]:
df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,14.275,2 hours ago,2021-12-30T12:59:01.380456Z,2021-12-30,15:39,"51.506321,-0.12714",Heavy Cloud,12.665,14.465
1,London,14.210,2 hours ago,2021-12-30T12:59:01.869683Z,2021-12-31,15:39,"51.506321,-0.12714",Light Rain,11.015,14.810


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,21.535,2 hours ago,2021-12-30T13:37:04.460044Z,2021-12-31,04:39,"-36.884109,174.770416",Heavy Cloud,16.885,22.74


# 🦄 Pro db 

In [ ]:
import sys
sys.path.append('../')

from prodb.core import generate_db, insert_row

### Initialise empty db

In [ ]:
dbpath = 'weather_db.csv'
cols = 'location temp high low weather_state'.split()

generate_db(cols=cols, dbpath=dbpath)
df = pd.read_csv(dbpath)
df.head()

✓💾 weather_db.csv (37 kb)


,location,temp,high,low,weather_state


## Add row

In [ ]:
dx = df_from_loc('Auckland')
dx

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,21.535,2 hours ago,2021-12-30T13:37:04.460044Z,2021-12-31,04:41,"-36.884109,174.770416",Heavy Cloud,16.885,22.74


In [ ]:
#export
def visit_city(df, cities, dbpath):
    if isinstance(cities, str): cities = [cities]
    for city in cities:
        dx = df_from_loc(city).round(1)

        data = {'location': dx.location.item(), 
                'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                'temp': dx.the_temp.item(),
                'high': dx.max_temp.item(),
                'low': dx.min_temp.item(),
                'weather_state': dx.weather_state_name.item(),
                'local_time': dx.local_time.item(),
                'latlong': dx.latt_long.item()}

        df = insert_row(df, data, dbpath)
    return df.round(1)

In [ ]:
df = visit_city(df, ['Christchurch', 'Wellington', 'Lagos', 'Zagreb'], dbpath)
display(df)

,location,temp,high,low,weather_state,latlong,local_time,time_utc
0,Christchurch,17.2,19.0,12.8,Heavy Rain,"-43.527519,172.635422",04:41,2021-12-30 15:41:55
1,Wellington,19.0,20.8,14.6,Heavy Cloud,"-41.283779,174.787979",04:41,2021-12-30 15:41:55
2,Lagos,32.3,34.2,24.1,Light Cloud,"6.439180,3.423480",16:41,2021-12-30 15:41:57
3,Zagreb,7.4,8.9,4.3,Light Rain,"45.807259,15.967600",16:41,2021-12-30 15:41:58


In [ ]:
df = visit_city(df, 'Yangon', dbpath)
display(df)

,location,temp,high,low,weather_state,latlong,local_time,time_utc
0,Christchurch,17.2,19.0,12.8,Heavy Rain,"-43.527519,172.635422",04:41,2021-12-30 15:41:55
1,Wellington,19.0,20.8,14.6,Heavy Cloud,"-41.283779,174.787979",04:41,2021-12-30 15:41:55
2,Lagos,32.3,34.2,24.1,Light Cloud,"6.439180,3.423480",16:41,2021-12-30 15:41:57
3,Zagreb,7.4,8.9,4.3,Light Rain,"45.807259,15.967600",16:41,2021-12-30 15:41:58
4,Yangon,30.5,32.0,20.9,Light Cloud,"16.803890,96.154694",22:12,2021-12-30 15:42:02


In [ ]:
df = visit_city(df, ['Singapore', 'Alexandria', 'Bangkok'], dbpath)
display(df)

,location,temp,high,low,weather_state,latlong,local_time,time_utc
0,Christchurch,17.2,19.0,12.8,Heavy Rain,"-43.527519,172.635422",04:41,2021-12-30 15:41:55
1,Wellington,19.0,20.8,14.6,Heavy Cloud,"-41.283779,174.787979",04:41,2021-12-30 15:41:55
2,Lagos,32.3,34.2,24.1,Light Cloud,"6.439180,3.423480",16:41,2021-12-30 15:41:57
3,Zagreb,7.4,8.9,4.3,Light Rain,"45.807259,15.967600",16:41,2021-12-30 15:41:58
4,Yangon,30.5,32.0,20.9,Light Cloud,"16.803890,96.154694",22:12,2021-12-30 15:42:02
5,Singapore,28.9,30.1,24.9,Heavy Rain,"1.293780,103.853256",23:42,2021-12-30 15:42:13
6,Alexandria,16.5,17.7,13.3,Light Rain,"31.210489, 29.912430",17:42,2021-12-30 15:42:14
7,Bangkok,32.4,32.1,25.0,Heavy Cloud,"13.753330,100.504822",22:42,2021-12-30 15:42:15
